In [39]:
import pyzx_param as zx
from fractions import Fraction
import random
from IPython.display import display, Markdown
import ipywidgets as widgets
import tsim
import numpy as np

The ZX calculus introduces two types of tensors in diagrammatic notation:

<img src="../figures/spiders.png" height=150px>

From this, we can define spiders that represent quantum gates:

<img src="../figures/gates.png" height=100px>

In [40]:
g = tsim.Circuit("CNOT 0 1").get_graph()
zx.draw(g)
g.normalize()

In [41]:
np.real(g.to_matrix()).astype(np.int32)

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0]], dtype=int32)

In [42]:
c = tsim.Circuit("""
    SQRT_X 1
    H 1
    T 1
    CNOT 0 1
    CNOT 1 0
    CNOT 0 1
    T 0
    H 0
""")

c.diagram("timeline-svg", height=160)

In [43]:
c.diagram("pyzx");

The ZX calculus introduces a set of rules that transform the diagram, but leave the underlying tensor invariant:

<img src="../figures/rules.png" height=400px>

The above rules are complete. From them, one can derive two rules that can be successively applied to reduce the number of vertices in any ZX diagram:

<img src="../figures/reduce.png" height=540px>

## Non-Clifford gates

When no Clifford gates are present, all vertices of a ZX diagram can be removed. Then we are just left with a scalar that represents a probability amplitude.

If non-Clifford gates are present, we cannot remove all vertices with the ZX rules. Instead, we can apply stabilizer rank decomposition to remove non-Clifford vertices at the cost of introducing a superposition of diagrams.

<img src="../figures/bss.png" alt="stabilizer rank" width=1200/>

In [44]:
def unfuse_non_cliffors(g):
    for v in list(g.vertices()):
        ty, p = g.type(v), g.phase(v)
        if p.denominator == 4:
            v1 = g.add_vertex(ty, qubit=-3, row=g.row(v), phase=Fraction(1, 4))
            g.add_to_phase(v, Fraction(-1, 4), set())
            g.add_edge((v, v1))


random.seed(3)
g = zx.generate.cliffordT(5, 200, p_t=0.1)
g.apply_effect("0" * 5)
g.apply_state("0" * 5)
zx.full_reduce(g)
g.normalize()
unfuse_non_cliffors(g)
zx.draw(g, show_scalar=True, scale=30)
print(f"Number of T-gates: {zx.tcount(g)}")

Number of T-gates: 10


In [45]:
def plotter(term, full_reduce=False):
    gsum = zx.simulate.replace_magic_states(g, pick_random=False)
    term_permuted = [4, 0, 1, 3, 2, 5, 6][term]
    gi = gsum.graphs[term_permuted]
    if full_reduce:
        zx.full_reduce(gi)
    zx.draw(gi, scale=30)
    display(Markdown(f"Number of T-gates: {zx.tcount(gi)}"))
    display(Markdown(f"Phase: {gi.scalar}"))


widgets.interactive(
    plotter,
    term=widgets.ToggleButtons(options=[0, 1, 2, 3, 4, 5, 6]),
    full_reduce=widgets.Checkbox(value=False),
)

interactive(children=(ToggleButtons(description='term', options=(0, 1, 2, 3, 4, 5, 6), value=0), Checkbox(valu…